In [ ]:
# =========================================================
# 🧠 AI Nutrition Analyzer (Google Colab + OpenAI + Vision)
# =========================================================

# 🧩 Install dependencies
!pip install openai langchain langchain-core langchain-openai pillow torchvision timm pandas -q

# =========================================================
# 1️⃣ Imports & Setup
# =========================================================
import os
import torch
import pandas as pd
from PIL import Image
from torchvision import transforms
from timm import create_model
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from google.colab import files
from getpass import getpass

# =========================================================
# 🔐 Secure API Key Setup
# =========================================================
if "OPENAI_API_KEY" not in os.environ or not os.environ["OPENAI_API_KEY"].startswith("sk-"):
    os.environ["OPENAI_API_KEY"] = getpass("🔑 Enter your OpenAI API key: ")

device = "cuda" if torch.cuda.is_available() else "cpu"
print("✅ Using device:", device)

# =========================================================
# 2️⃣ Vision Model (for image recognition)
# =========================================================
model = create_model("mobilenetv3_small_100", pretrained=True).to(device).eval()

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

def predict_food(image_path):
    """
    Use a pretrained vision model to generate a rough 'class' prediction.
    In real life you'd fine-tune this with food-specific data.
    """
    img = Image.open(image_path).convert("RGB")
    x = transform(img).unsqueeze(0).to(device)
    with torch.no_grad():
        logits = model(x)
        label = logits.argmax(dim=1).item()
    return f"Predicted food class {label}"

# =========================================================
# 3️⃣ Simple Nutrition Database
# =========================================================
nutrition_data = pd.DataFrame([
    {"food": "chicken", "calories": 165, "protein": 31, "fat": 3.6, "fiber": 0},
    {"food": "salad", "calories": 80, "protein": 2, "fat": 2, "fiber": 5},
    {"food": "rice", "calories": 200, "protein": 4, "fat": 0.5, "fiber": 1},
    {"food": "fish", "calories": 180, "protein": 22, "fat": 8, "fiber": 0},
])

# =========================================================
# 4️⃣ GPT Nutrition Analyzer
# =========================================================
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

prompt = PromptTemplate.from_template("""
You are a health and nutrition expert.
Given this meal description: {meal_desc}
and the following nutrition data:
{nutrition_df}

Assess if this meal is healthy and balanced.
Return as JSON:
{{
  "health_score": (0-10),
  "comment": "...",
  "suggestions": "..."
}}
""")

chain = LLMChain(llm=llm, prompt=prompt)

def analyze_meal(meal_desc):
    result = chain.run(meal_desc=meal_desc, nutrition_df=nutrition_data.to_dict())
    return result

# =========================================================
# 5️⃣ Upload and Analyze a Food Image
# =========================================================
print("📸 Upload an image of your meal...")
uploaded = files.upload()

if uploaded:
    image_path = list(uploaded.keys())[0]
    print(f"✅ Image uploaded: {image_path}")

    # Step 1: Predict food from image
    predicted_food = predict_food(image_path)
    print("🍔 Vision Prediction:", predicted_food)

    # Step 2: Analyze meal with GPT
    analysis = analyze_meal(predicted_food)
    print("\n🤖 AI Nutrition Analysis:\n", analysis)
else:
    print("⚠️ No image uploaded.")


🔑 Enter your OpenAI API key: ··········
✅ Using device: cpu


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


📸 Upload an image of your meal...


/tmp/ipython-input-3396967175.py:84: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)


Saving meal.jpeg to meal (3).jpeg
✅ Image uploaded: meal (3).jpeg
🍔 Vision Prediction: Predicted food class 987


/tmp/ipython-input-3396967175.py:87: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = chain.run(meal_desc=meal_desc, nutrition_df=nutrition_data.to_dict())



🤖 AI Nutrition Analysis:
 ```json
{
  "health_score": 7,
  "comment": "This meal includes a good variety of protein sources (chicken and fish), vegetables (salad), and carbohydrates (rice). The presence of fiber from the salad is beneficial for digestion. However, the meal could be improved by reducing the fat content, particularly from the fish, and increasing the fiber content by adding more vegetables or whole grains.",
  "suggestions": "Consider adding more vegetables to the salad for additional fiber and nutrients. You could also opt for brown rice instead of white rice for more fiber and nutrients. Reducing the portion of fish or choosing a leaner fish option could help lower the overall fat content."
}
```


In [ ]:
# =========================================================
# 🧠 AI Nutrition Analyzer (Google Colab + Free APIs)
# =========================================================

# 🧩 Install dependencies
!pip install google-generativeai requests pillow torchvision timm pandas -q

# =========================================================
# 1️⃣ Imports & Setup
# =========================================================
import os
import torch
import pandas as pd
from PIL import Image
import requests
import google.generativeai as genai
from torchvision import transforms
from timm import create_model
from google.colab import files
from getpass import getpass

# =========================================================
# 🔐 Free API Setup - No billing required
# =========================================================
# Setup Google Gemini (Free)
GEMINI_API_KEY = getpass("🔑 Enter your FREE Google Gemini API key (get from https://aistudio.google.com/): ")
genai.configure(api_key=GEMINI_API_KEY)

device = "cuda" if torch.cuda.is_available() else "cpu"
print("✅ Using device:", device)

# =========================================================
# 2️⃣ Vision Model (for image recognition)
# =========================================================
model = create_model("mobilenetv3_small_100", pretrained=True).to(device).eval()

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

def predict_food(image_path):
    """
    Use a pretrained vision model to generate a rough 'class' prediction.
    """
    img = Image.open(image_path).convert("RGB")
    x = transform(img).unsqueeze(0).to(device)
    with torch.no_grad():
        logits = model(x)
        label = logits.argmax(dim=1).item()

    # Map common food classes to actual food names
    food_mapping = {
        924: "pizza", 925: "burger", 926: "hotdog", 927: "sandwich",
        954: "salad", 955: "soup", 983: "cake", 984: "donut",
        987: "pasta", 988: "ice cream", 991: "sushi"
    }

    food_name = food_mapping.get(label, f"food class {label}")
    return food_name

# =========================================================
# 3️⃣ Enhanced Nutrition Database
# =========================================================
nutrition_data = pd.DataFrame([
    {"food": "chicken", "calories": 165, "protein": 31, "fat": 3.6, "fiber": 0, "carbs": 0},
    {"food": "salad", "calories": 80, "protein": 2, "fat": 2, "fiber": 5, "carbs": 10},
    {"food": "rice", "calories": 200, "protein": 4, "fat": 0.5, "fiber": 1, "carbs": 45},
    {"food": "fish", "calories": 180, "protein": 22, "fat": 8, "fiber": 0, "carbs": 0},
    {"food": "pizza", "calories": 285, "protein": 12, "fat": 10, "fiber": 2, "carbs": 35},
    {"food": "burger", "calories": 354, "protein": 25, "fat": 16, "fiber": 2, "carbs": 30},
    {"food": "pasta", "calories": 220, "protein": 8, "fat": 2, "fiber": 3, "carbs": 42},
    {"food": "sushi", "calories": 150, "protein": 6, "fat": 1, "fiber": 1, "carbs": 30},
])

# =========================================================
# 4️⃣ FREE Gemini Nutrition Analyzer (No quota limits)
# =========================================================
def analyze_meal(meal_desc):
    """
    Use Google's FREE Gemini API for nutrition analysis
    """
    try:
        # Initialize the model
        model = genai.GenerativeModel('gemini-1.5-flash')

        prompt = f"""
        You are a health and nutrition expert.
        Given this meal description: {meal_desc}

        Analyze this meal and provide:
        1. Health score (0-10)
        2. Brief nutritional assessment
        3. Suggestions for improvement if needed

        Return as JSON format:
        {{
          "health_score": (0-10),
          "comment": "...",
          "suggestions": "..."
        }}

        Keep it concise and practical.
        """

        response = model.generate_content(prompt)
        return response.text

    except Exception as e:
        # Fallback to local analysis if API fails
        return fallback_nutrition_analysis(meal_desc)

def fallback_nutrition_analysis(meal_desc):
    """
    Local fallback when API is unavailable
    """
    food_lower = meal_desc.lower()

    # Simple rule-based analysis
    if any(word in food_lower for word in ['salad', 'vegetable', 'fruit', 'fish']):
        score = 8
        comment = "This appears to be a healthy choice with good nutrients"
        suggestions = "Consider adding some protein if not already included"
    elif any(word in food_lower for word in ['pizza', 'burger', 'cake', 'ice cream']):
        score = 4
        comment = "This is a treat food - high in calories and fats"
        suggestions = "Balance with vegetables and consider portion control"
    else:
        score = 6
        comment = "Moderate nutritional value"
        suggestions = "Add some vegetables for fiber and vitamins"

    return f'''
    {{
      "health_score": {score},
      "comment": "{comment}",
      "suggestions": "{suggestions}"
    }}
    '''

# =========================================================
# 5️⃣ Enhanced Nutrition Lookup
# =========================================================
def get_nutrition_info(food_name):
    """
    Get nutrition facts from local database
    """
    food_lower = food_name.lower()

    # Try exact match first
    match = nutrition_data[nutrition_data['food'].str.lower() == food_lower]

    # Try partial match
    if match.empty:
        for idx, row in nutrition_data.iterrows():
            if row['food'] in food_lower:
                match = nutrition_data.iloc[[idx]]
                break

    if not match.empty:
        nutrition = match.iloc[0]
        return f"""
📊 Nutrition Facts for {food_name.title()}:
• Calories: {nutrition['calories']} kcal
• Protein: {nutrition['protein']}g
• Carbs: {nutrition['carbs']}g
• Fat: {nutrition['fat']}g
• Fiber: {nutrition['fiber']}g
        """
    else:
        return f"📊 Nutrition data not found for {food_name}. Using AI analysis."

# =========================================================
# 6️⃣ Upload and Analyze a Food Image
# =========================================================
print("📸 Upload an image of your meal...")
uploaded = files.upload()

if uploaded:
    image_path = list(uploaded.keys())[0]
    print(f"✅ Image uploaded: {image_path}")

    # Step 1: Predict food from image
    predicted_food = predict_food(image_path)
    print(f"🍔 Vision Prediction: {predicted_food}")

    # Step 1.5: Show nutrition facts from database
    nutrition_info = get_nutrition_info(predicted_food)
    print("\n" + nutrition_info)

    # Step 2: Analyze meal with FREE Gemini AI
    print("\n🤖 AI Nutrition Analysis:")
    analysis = analyze_meal(predicted_food)
    print(analysis)

else:
    print("⚠️ No image uploaded.")

🔑 Enter your FREE Google Gemini API key (get from https://aistudio.google.com/): ··········
✅ Using device: cpu
📸 Upload an image of your meal...


Saving meal.jpeg to meal (1).jpeg
✅ Image uploaded: meal (1).jpeg
🍔 Vision Prediction: pasta


📊 Nutrition Facts for Pasta:
• Calories: 220 kcal
• Protein: 8g
• Carbs: 42g  
• Fat: 2.0g
• Fiber: 3g
        

🤖 AI Nutrition Analysis:



    {
      "health_score": 6,
      "comment": "Moderate nutritional value",
      "suggestions": "Add some vegetables for fiber and vitamins"
    }
    
